In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import os
import warnings
from selenium import webdriver
from glob import glob
import datetime
import time
warnings.filterwarnings('ignore')

c:\users\сергей\appdata\local\programs\python\python38\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [59]:
os.chdir('Classic')

Запускаем GoogleChrome для обработки запросов

In [3]:
exe_path = 'C:\\Users\\Сергей\\Desktop\\chromedriver.exe'

In [4]:
driver = webdriver.Chrome(executable_path=exe_path)

In [7]:
def prepare_dataset(request,type_, folder_name,PATH):
    initial_PATH = PATH
    PATH_full = os.path.join(PATH,type_)
    try:
        os.mkdir(os.path.join(PATH_full,folder_name))
    except:
        pass
    request = request.split()
    request = '%20'.join(request)
    driver.get(f'https://yandex.ru/images/search?text={request}&from=tabbar')
    
    SCROLL_PAUSE_TIME = 30

    # Получаем станицу после одного пролистывания
    last_height = driver.execute_script("return document.body.scrollHeight")
    init_time = datetime.datetime.now()
    
    res_PATH = os.path.join(PATH_full,folder_name)
    
    while True:
        # Прокручиваем страницу вниз
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Ждем загрузки страницы
        time.sleep(SCROLL_PAUSE_TIME)
        finish_time = datetime.datetime.now()
        
        if (finish_time - init_time).seconds > 300:
            break

    page = driver.page_source
    os.chdir(os.path.join(PATH_full,folder_name))
    soup = BeautifulSoup(page,'html.parser')
    images = soup.find_all('img')

    #получение файлов и их сохранение локально
    for num, image in enumerate(images):
        name = f'{num}'
        try:
            link = image['src']
        except:
            pass
        try:
            with open(f'{name}.jpg','wb') as f:
                im = requests.get('https:'+ link)
                f.write(im.content)
        except:
            try:
                im = requests.get(link)
                f.write(im.content)
            except:
                try:
                    with open(f'{name}.jpg','wb') as f:
                        im = requests.get('http:'+ link)
                        f.write(im.content)
                except:
                    pass
                    
    
    
    os.chdir(initial_PATH)
    return len(glob(f'{res_PATH}\\*.jpg'))


In [8]:
prepare_dataset('здания в стиле классицизм','Buildings', 'Классицизм', os.getcwd())

1484